<a href="https://colab.research.google.com/github/MariusMC8/Thesis-coding/blob/main/Risk_ranking_maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geopandas matplotlib

In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
url = "https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_ESP_1.json"
regions = gpd.read_file(url)

# Climate indicator: Hot Days above 35C (Climate-ADAPT)

In [62]:
from google.colab import files
uploaded = files.upload()

Saving Template storage data to analyse.csv to Template storage data to analyse.csv


In [63]:
import os

old_name = "Template storage data to analyse.csv"
new_name = "Hot_days_35C.csv"

os.rename(old_name, new_name)

In [64]:
data = pd.read_csv("Hot_days_35C.csv", sep=';', decimal=",", encoding='latin1')
data.head()

,ï»¿ID,Coordinates,Storage type,Region,Status,Power (MW),Capacity (MWh),Duration (hours),Importance score
0,Malpica Wind plus hybrid storage,"43.320000 , -8.810000",Battery,Galicia,Announced,7.75,15.5,2.00,2
1,BESS Sanxenxo,"42.400000 , -8.810000",Battery,Galicia,Announced,5.00,20.0,4.00,2
2,Santiago-Jares,"42.390000 , -7.090000",Battery,Galicia,Operational,5.00,5.0,1.00,2
3,Conso,"42.250000 , -7.160000",PHS,Galicia,Operational,270.00,2280.0,8.44,3
4,Soutelo PSP,"42.150000 , -7.190000",PHS,Galicia,Operational,215.00,NaN,NaN,3


In [66]:
#Fix region names
data.columns = data.columns.str.strip()
data["Region"].unique()

array(['Galicia', 'PrincipadodeAsturias', 'Cantabria', 'PaÃ\xadsVasco',
       'ComunidadForaldeNavarra', 'AragÃ³n', 'CataluÃ±a', 'IslasBaleares',
       'ComunidadValenciana', 'RegiÃ³ndeMurcia', 'AndalucÃ\xada',
       'CastillayLeÃ³n', 'CeutayMelilla', 'IslasCanarias', 'Extremadura',
       'Castilla-LaMancha'], dtype=object)

In [67]:
region_fix = {
    "PaÃ\xadsVasco": "PaísVasco",
    "AragÃ³n": "Aragón",
    "CataluÃ±a": "Cataluña",
    "RegiÃ³ndeMurcia": "RegióndeMurcia",
    "AndalucÃ\xada": "Andalucía",
    "CastillayLeÃ³n": "CastillayLeón"}

data["Region"] = data["Region"].replace(region_fix)

data["Region"].unique()

array(['Galicia', 'PrincipadodeAsturias', 'Cantabria', 'PaísVasco',
       'ComunidadForaldeNavarra', 'Aragón', 'Cataluña', 'IslasBaleares',
       'ComunidadValenciana', 'RegióndeMurcia', 'Andalucía',
       'CastillayLeón', 'CeutayMelilla', 'IslasCanarias', 'Extremadura',
       'Castilla-LaMancha'], dtype=object)

In [68]:
#Hot days above 35°C 1991-2020 for each region
hot_days_35_present_dict = {
    "Galicia": 0.19,
    "PrincipadodeAsturias": 0.0,
    "Cantabria": 0.05,
    "PaísVasco": 0.38,
    "ComunidadForaldeNavarra": 4.83,
    "Aragón": 8.94,
    "Cataluña": 2.92,
    "ComunidadValenciana": 4.31,
    "RegióndeMurcia": 8.47,
    "Andalucía": 20.73,
    "Castilla-LaMancha": 18.91,
    "CastillayLeón": 3.58,
    "Extremadura": 26.69,
    "IslasCanarias": 0.0,
    "IslasBaleares": 0.26,
    "CeutayMelilla": 3.84,
}

# Clean column names by stripping whitespace
data.columns = data.columns.str.strip()

# Add a new column based on region
data["Hot days >35C 1991-2020"] = data["Region"].map(hot_days_35_present_dict)

In [69]:
#Define bins for hazard strength
bins = [0, 20, float("inf")]
labels = [1, 2]

data["Hot days 35C 1991-2020 strength"] = pd.cut(
    data["Hot days >35C 1991-2020"].fillna(0), # Fill NaN values with 0 before cutting
    bins=bins,
    labels=labels,
    include_lowest=True
).astype(int)

In [70]:
print(data[["Region", "Hot days >35C 1991-2020", "Hot days 35C 1991-2020 strength"]])

                Region  Hot days >35C 1991-2020  \
0              Galicia                     0.19   
1              Galicia                     0.19   
2              Galicia                     0.19   
3              Galicia                     0.19   
4              Galicia                     0.19   
..                 ...                      ...   
128  Castilla-LaMancha                    18.91   
129  Castilla-LaMancha                    18.91   
130  Castilla-LaMancha                    18.91   
131  Castilla-LaMancha                    18.91   
132  Castilla-LaMancha                    18.91   

     Hot days 35C 1991-2020 strength  
0                                  1  
1                                  1  
2                                  1  
3                                  1  
4                                  1  
..                               ...  
128                                1  
129                                1  
130                                1

In [71]:
#Vulnerability score
vulnerability_hot_days_35 = {
    "Battery": 3,
    "PHS": 2,
    "Flywheel": 1
}

data["Vulnerability hot days 35C"] = data["Storage type"].map(vulnerability_hot_days_35)

In [72]:
data.head()

,ï»¿ID,Coordinates,Storage type,Region,Status,Power (MW),Capacity (MWh),Duration (hours),Importance score,Hot days >35C 1991-2020,Hot days 35C 1991-2020 strength,Vulnerability hot days 35
0,Malpica Wind plus hybrid storage,"43.320000 , -8.810000",Battery,Galicia,Announced,7.75,15.5,2.00,2,0.19,1,3.0
1,BESS Sanxenxo,"42.400000 , -8.810000",Battery,Galicia,Announced,5.00,20.0,4.00,2,0.19,1,3.0
2,Santiago-Jares,"42.390000 , -7.090000",Battery,Galicia,Operational,5.00,5.0,1.00,2,0.19,1,3.0
3,Conso,"42.250000 , -7.160000",PHS,Galicia,Operational,270.00,2280.0,8.44,3,0.19,1,2.0
4,Soutelo PSP,"42.150000 , -7.190000",PHS,Galicia,Operational,215.00,NaN,NaN,3,0.19,1,2.0


In [ ]:
#Impact of climate change on the climate hazard (step 4)